<a href="https://colab.research.google.com/github/nitron-alpha-kplr/Reinforcement-Learning/blob/main/Frozen_Lake_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q* Learning with FrozenLake 4x4

In this Notebook, we'll implement an agent <b>that plays FrozenLake.</b>

![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

The goal of this game is <b>to go from the starting state (S) to the goal state (G)</b> by walking only on frozen tiles (F) and avoid holes (H). However, the ice is slippery, **so you won't always move in the direction you intend (stochastic environment)**



## Prerequisites 🏗️
Before diving on the notebook **you need to understand**:
- The foundations of Reinforcement learning (MC, TD, Rewards hypothesis...) [Article](https://medium.freecodecamp.org/an-introduction-to-reinforcement-learning-4339519de419)
- Q-learning [Article](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe)
- In the [video version](https://www.youtube.com/watch?v=q2ZOEFAaaI0)  we implemented a Q-learning agent that learns to play OpenAI Taxi-v2 🚕 with Numpy.

## Step -1: Install the dependencies on Google Colab

In [1]:
!pip install numpy
!pip install gym==0.7.4.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.0/841.0 kB 22.4 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.7.4-py3-none-any.whl size=204668 sha256=f5b1fdc1e7c4196ecf223036a9a17614882b34f0dc22d1a27e4a0a1fcc4ca4d4
  Stored in directory: /root/.cache/pip/wheels/de/55/26/d9b9f975bc80a53913520dd7ded672660cf82a01ac3b0a8769
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [2]:
#import the required libraries.
import numpy as np
import gym
import random

In [3]:
#create the environment usign OpenAI Gym
env = gym.make("FrozenLake-v0")

INFO:gym.envs.registration:Making new env: FrozenLake-v0
[2023-06-21 08:40:45,564] Making new env: FrozenLake-v0
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Step 2: Create the Q-table and initialize it

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n

print(f"Action Space : {action_size} | State Space: {state_size}")

Action Space : 4 | State Space: 16


In [5]:
qtable = np.zeros((state_size, action_size))
print(qtable.shape)

(16, 4)


## Step 3: Create Required Hyperparameters

In [6]:
total_episodes = 15000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005             # Exponential decay rate for exploration prob

## Step 4 : Q-Learning Algorithm

In [10]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward

        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done == True:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.49233333333333335
[[6.00408381e-02 5.91115181e-02 8.70003306e-02 5.66060325e-02]
 [2.52924772e-03 3.34681333e-04 2.01480643e-03 5.61592500e-02]
 [1.07964890e-02 5.81280534e-03 3.78744639e-03 3.74962625e-02]
 [6.11642581e-03 7.67178933e-03 1.10507261e-03 2.88860537e-02]
 [8.34609630e-02 1.74158548e-02 6.08782785e-02 5.31466424e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.19570067e-01 2.25037160e-08 4.59492161e-04 6.64359177e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.18891819e-02 6.36804260e-03 8.31764996e-03 8.21006006e-02]
 [2.86559310e-02 4.22790463e-01 4.52612896e-02 7.52831845e-03]
 [7.83334339e-01 6.10289406e-03 8.41369418e-03 5.08412370e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.80330515e-02 2.39660369e-02 6.83358550e-01 1.32131798e-02]
 [2.70049906e-01 9.98025455e-01 4.30850915e-01 2.29469735e-01]
 [0.00000000e+00 0

In [8]:
for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ",episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, done, info = env.step(action)

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()

            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 21
****************************************************
EPISODE  1
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 14
****************************************************
EPISODE  2
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 37
****************************************************
EPISODE  3
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 15
****************************************************
EPISODE  4
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 18
